# Porting Human-GEM (Robinson et al 2020), stage 1


Objective: To port the metabolic model in Robinson et al (2020) for use in mummichog and Azimuth.

This is a two stage process:

1. porting out compounds and reactions (refer to data structures in https://github.com/shuzhao-li/azimuth_metabolomics), save to Pickle obj, which is uploaded to Azimuth DB separately.

2. Building index and full model for mummichog use, from stage 1.

Watch out:

* Do not use chemical formula in these "GEMs" without verification. They usually have chemical formula in charged state, which can't be used for the neutral ion in mass spec data.

* Identifiers are the main issue. We may have to look up by names via RefMet.


Model retrieved on 2020-03-31 from

https://github.com/SysBioChalmers/Human-GEM

To start Docker for notebook:

shuzhao@canyon:~/projects/Azimuth/model_porting$ docker run -v /home/shuzhao/projects/Azimuth/model_porting:/home/jovyan/ -p 8888:8888 jupyter/scipy-notebook


By Shuzhao Li, 2020-04-08

Note 2021-07-22 added to mass2chem: mostly a messy record of mapping identifiers across models and fields.


In [1]:
# tool to parse XML
import xml.etree.ElementTree as ET
# tool to parse Excel xlsx
import xlrd

xmlFile = 'Human-GEM/ModelFiles/xml/HumanGEM.xml'
excelFile = 'Human-GEM/ModelFiles/xlsx/HumanGEM.xlsx'

# Need RefMet for name mapping. Retrived 2020-03-29 from Metabolomics Workbench
refmetFile = 'refmet.csv'

# parse XML
tree = ET.parse(xmlFile)
root = tree.getroot()
for child in root: print (child)

<Element '{http://www.sbml.org/sbml/level3/version1/core}model' at 0x7f9476689b30>


In [2]:
# Find lower level structure in this XML file
model = root.find('{http://www.sbml.org/sbml/level3/version1/core}model')
for child in model: print (child)

<Element '{http://www.sbml.org/sbml/level3/version1/core}notes' at 0x7f9476689c50>
<Element '{http://www.sbml.org/sbml/level3/version1/core}annotation' at 0x7f9476689ef0>
<Element '{http://www.sbml.org/sbml/level3/version1/core}listOfUnitDefinitions' at 0x7f9476696890>
<Element '{http://www.sbml.org/sbml/level3/version1/core}listOfCompartments' at 0x7f9476696bf0>
<Element '{http://www.sbml.org/sbml/level3/version1/core}listOfSpecies' at 0x7f947669a110>
<Element '{http://www.sbml.org/sbml/level3/version1/core}listOfParameters' at 0x7f9474322ef0>
<Element '{http://www.sbml.org/sbml/level3/version1/core}listOfReactions' at 0x7f947432a170>
<Element '{http://www.sbml.org/sbml/level3/version1/fbc/version2}listOfObjectives' at 0x7f945df2acb0>
<Element '{http://www.sbml.org/sbml/level3/version1/fbc/version2}listOfGeneProducts' at 0x7f945df2afb0>
<Element '{http://www.sbml.org/sbml/level3/version1/groups/version1}listOfGroups' at 0x7f945c817bf0>


In [3]:
for child in model.find('{http://www.sbml.org/sbml/level3/version1/core}listOfCompartments'): print (child.attrib)

{'metaid': 's', 'sboTerm': 'SBO:0000290', 'id': 's', 'name': 'Extracellular', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'p', 'sboTerm': 'SBO:0000290', 'id': 'p', 'name': 'Peroxisome', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'm', 'sboTerm': 'SBO:0000290', 'id': 'm', 'name': 'Mitochondria', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'c', 'sboTerm': 'SBO:0000290', 'id': 'c', 'name': 'Cytosol', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'l', 'sboTerm': 'SBO:0000290', 'id': 'l', 'name': 'Lysosome', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'r', 'sboTerm': 'SBO:0000290', 'id': 'r', 'name': 'Endoplasmic reticulum', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'g', 'sboTerm': 'SBO:0000290', 'id': 'g', 'name': 'Golgi apparatus', 'spatialDimensions': '3', 'size': '1', 'constant': 'true'}
{'metaid': 'n', 'sboTerm': 'SBO:0000290', 'id'

## Getting species/metabolites

In [4]:
# to get species 
species = model.find('{http://www.sbml.org/sbml/level3/version1/core}listOfSpecies')
print(len(species))
for c in species[:5]: print ('\n', c.attrib)
    
# add species to new dict
cpd_dict = {}
for c in species:
    if c.attrib:
        try:
            cpd_dict[ c.attrib['id'] ] = c.attrib 
        except KeyError:
            print (c.attrib)
            
print("\n\nGot %d of compounds..." %len(cpd_dict))
print( list(cpd_dict.values())[:2] )

10073

 {'metaid': 'M_m00001c', 'sboTerm': 'SBO:0000247', 'id': 'M_m00001c', 'name': '(-)-trans-carveol', 'compartment': 'c', 'initialConcentration': '0', 'hasOnlySubstanceUnits': 'false', 'boundaryCondition': 'false', 'constant': 'false', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge': '0', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula': 'C10H16O'}

 {'metaid': 'M_m00001s', 'sboTerm': 'SBO:0000247', 'id': 'M_m00001s', 'name': '(-)-trans-carveol', 'compartment': 's', 'initialConcentration': '0', 'hasOnlySubstanceUnits': 'false', 'boundaryCondition': 'false', 'constant': 'false', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge': '0', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula': 'C10H16O'}

 {'metaid': 'M_m00002c', 'sboTerm': 'SBO:0000247', 'id': 'M_m00002c', 'name': '(+)-alpha-pinene', 'compartment': 'c', 'initialConcentration': '0', 'hasOnlySubstanceUnits': 'false', 'boundaryCondition': 'false', 'con

In [5]:
# how many unique metabolites/compounds?

unique_ids = [x[2:8] for x in cpd_dict.keys()]
print( len(unique_ids), len(set(unique_ids)) )

10073 4061


So there are 4061 metabolites in this model, inflated to 10073 due to compartmentalization.

Metabolite identifiers will be tricky. Let's get reactions and come back to metabolites.

## Getting reactions

In [6]:
# to get reactions
rxns = model.find('{http://www.sbml.org/sbml/level3/version1/core}listOfReactions')
print (len(rxns))
for r in rxns[1:3]: 
    print ('\n', r.attrib)
    for x in r.find('{http://www.sbml.org/sbml/level3/version1/core}listOfReactants'):
        print(x.attrib)

13097

 {'metaid': 'R_HMR_3907', 'sboTerm': 'SBO:0000176', 'id': 'R_HMR_3907', 'reversible': 'false', 'fast': 'false', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}lowerFluxBound': 'FB2N0', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}upperFluxBound': 'FB3N1000'}
{'species': 'M_m01796c', 'stoichiometry': '1', 'constant': 'true'}
{'species': 'M_m02554c', 'stoichiometry': '1', 'constant': 'true'}

 {'metaid': 'R_HMR_4097', 'sboTerm': 'SBO:0000176', 'id': 'R_HMR_4097', 'reversible': 'false', 'fast': 'false', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}lowerFluxBound': 'FB2N0', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}upperFluxBound': 'FB3N1000'}
{'species': 'M_m01252c', 'stoichiometry': '1', 'constant': 'true'}
{'species': 'M_m01371c', 'stoichiometry': '1', 'constant': 'true'}
{'species': 'M_m01597c', 'stoichiometry': '1', 'constant': 'true'}


Pathway assignment? Not in here? 

**Check the Excel file**

In [13]:
xlsxData = xlrd.open_workbook(excelFile)
sheets = xlsxData.sheets()
for x in sheets: print(x.name)

RXNS
METS
COMPS
GENES
MODEL


In [12]:
dir(sheets[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cell_attr_to_xfx',
 '_cell_types',
 '_cell_values',
 '_cell_xf_indexes',
 '_dimncols',
 '_dimnrows',
 '_first_full_rowx',
 '_ixfe',
 '_maxdatacolx',
 '_maxdatarowx',
 '_position',
 '_repr_these',
 '_xf_index_stats',
 '_xf_index_to_xl_type_map',
 'automatic_grid_line_colour',
 'bf',
 'biff_version',
 'book',
 'bt',
 'cached_normal_view_mag_factor',
 'cached_page_break_preview_mag_factor',
 'cell',
 'cell_note_map',
 'cell_type',
 'cell_value',
 'cell_xf_index',
 'col',
 'col_label_ranges',
 'col_slice',
 'col_types',
 'col_values',
 'colinfo_map',
 'columns_from_right_to_left',
 'computed_column_width',
 'cooked_normal_v

In [16]:
for ii in range(5):
    print(sheets[1].row_values(ii))

['#', 'ID', 'NAME', 'UNCONSTRAINED', 'MIRIAM', 'COMPOSITION', 'InChI', 'COMPARTMENT', 'REPLACEMENT ID', 'CHARGE']
['', '(-)-trans-carveol[c]', '(-)-trans-carveol', '', '', 'C10H16O', '', 'c', 'm00001c', 0.0]
['', '(-)-trans-carveol[s]', '(-)-trans-carveol', '', '', 'C10H16O', '', 's', 'm00001s', 0.0]
['', '(+)-alpha-pinene[c]', '(+)-alpha-pinene', '', '', 'C10H16', '', 'c', 'm00002c', 0.0]
['', '(+)-alpha-pinene[s]', '(+)-alpha-pinene', '', '', 'C10H16', '', 's', 'm00002s', 0.0]


In [17]:
for ii in range(5):
    print(sheets[0].row_values(ii))

['#', 'ID', 'NAME', 'EQUATION', 'EC-NUMBER', 'GENE ASSOCIATION', 'LOWER BOUND', 'UPPER BOUND', 'OBJECTIVE', 'COMPARTMENT', 'MIRIAM', 'SUBSYSTEM', 'REPLACEMENT ID', 'NOTE', 'REFERENCE', 'CONFIDENCE SCORE']
['', 'HMR_3905', '', 'ethanol[c] + NAD+[c] => acetaldehyde[c] + H+[c] + NADH[c]', 'EC:1.1.1.1; EC:1.1.1.71', 'ENSG00000147576 or ENSG00000172955 or ENSG00000180011 or ENSG00000187758 or ENSG00000196344 or ENSG00000196616 or ENSG00000197894 or ENSG00000198099 or ENSG00000248144', '', '', '', '', '', 'Glycolysis / Gluconeogenesis', '', '', 'PMID:10868354;PMID:12491384;PMID:12818203;PMID:14674758;PMID:15289102;PMID:15299346;PMID:15327949;PMID:15682493;PMID:15713978', 0.0]
['', 'HMR_3907', '', 'ethanol[c] + NADP+[c] => acetaldehyde[c] + H+[c] + NADPH[c]', 'EC:1.1.1.2', 'ENSG00000117448', '', '', '', '', '', 'Glycolysis / Gluconeogenesis', '', '', '', 0.0]
['', 'HMR_4097', '', 'acetate[c] + ATP[c] + CoA[c] => acetyl-CoA[c] + AMP[c] + PPi[c]', 'EC:6.2.1.1', 'ENSG00000131069', '', '', '', ''

In [18]:
header = sheets[0].row_values(0)
header.index("SUBSYSTEM")

11

In [19]:
# taking SUBSYSTEM as pathway
pathways = [sheets[0].row_values(ii)[11] for ii in range(1, sheets[0].nrows)]
print( len(pathways), len(set(pathways)) )

13417 147


In [21]:
pathways = list(set(pathways))
pathways.sort()
print(pathways)

['Acyl-CoA hydrolysis', 'Acylglycerides metabolism', 'Alanine, aspartate and glutamate metabolism', 'Alkaloids Biosynthesis', 'Amino sugar and nucleotide sugar metabolism', 'Aminoacyl-tRNA biosynthesis', 'Androgen metabolism', 'Arachidonic acid metabolism', 'Arginine and proline metabolism', 'Artificial reactions', 'Ascorbate and aldarate metabolism', 'Beta oxidation of branched-chain fatty acids (mitochondrial)', 'Beta oxidation of di-unsaturated fatty acids (n-6) (mitochondrial)', 'Beta oxidation of di-unsaturated fatty acids (n-6) (peroxisomal)', 'Beta oxidation of even-chain fatty acids (mitochondrial)', 'Beta oxidation of even-chain fatty acids (peroxisomal)', 'Beta oxidation of odd-chain fatty acids (mitochondrial)', 'Beta oxidation of odd-chain fatty acids (peroxisomal)', 'Beta oxidation of phytanic acid (peroxisomal)', 'Beta oxidation of poly-unsaturated fatty acids (mitochondrial)', 'Beta oxidation of unsaturated fatty acids (n-7) (mitochondrial)', 'Beta oxidation of unsaturat

## To get rxns from SBML, but

* EC numbers for enzymes from the xlsx file

* Pathway association from the xlsx file (SUBSYSTEM)

Reactants and products are better defined in the SBML file.

But need to map metabolite IDs to that free of compartmentalization


## Compiling the metabolite list

* use Refmet to match to PubChem ID if possible


In [29]:
# Refmet is in csv format

import csv

refmetDict = {}
with open(refmetFile, newline='') as csvfile:
    myreader = csv.reader(csvfile, delimiter=',')
    for row in myreader:
        try:
            refmetDict[row[0]] = row
        except IndexError:
            print(len(refmetDict))
            pass


95385


In [32]:
print(len(refmetDict), 
list(refmetDict.values())[:2]
    )

print( refmetDict[' refmet_name'] )

95385 [[' refmet_name', 'super_class', 'main_class', 'sub_class', 'formula', 'exactmass', 'inchi_key', 'pubchem_cid'], ['Anatoxin A', 'Alkaloids', 'Aliphatic heteropolycyclic compounds', 'Anatoxins', 'C10H15NO', '165.1154', 'SGNXVBOIDPPRJJ-UHFFFAOYSA-N', '431734']]
[' refmet_name', 'super_class', 'main_class', 'sub_class', 'formula', 'exactmass', 'inchi_key', 'pubchem_cid']


In [33]:
print(refmetDict['trans-carveol'])

KeyError: 'trans-carveol'

In [36]:
#
# (-)-trans-carveol[s] in the GEM model is Trans-carveol in Refmet, etc
#

lowrefmetDict = {}
for k,v in refmetDict.items():
    lowrefmetDict[k.lower()] = v
    
print(len(lowrefmetDict))
print(lowrefmetDict['trans-carveol'])

95384
['Trans-carveol', 'Prenol Lipids', 'Isoprenoids', 'C10 isoprenoids', 'C10H16O', '152.120115', 'BAVONGHXFVOKBV-VHSXEESVSA-N', '443178']


In [37]:
print(len(cpd_dict))
print(cpd_dict['M_m00001c'])

10073
{'metaid': 'M_m00001c', 'sboTerm': 'SBO:0000247', 'id': 'M_m00001c', 'name': '(-)-trans-carveol', 'compartment': 'c', 'initialConcentration': '0', 'hasOnlySubstanceUnits': 'false', 'boundaryCondition': 'false', 'constant': 'false', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge': '0', '{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula': 'C10H16O'}


In [39]:
new_cpd_dict = {}
for k in cpd_dict:
    new_cpd_dict[k[:-1]] = (cpd_dict[k]['name'], cpd_dict[k]['id'])
    
print(len(new_cpd_dict), k, new_cpd_dict[k[:-1]])

4141 M_m10042c ('ursodeoxycholoyl-CoA', 'M_m10042c')


In [43]:
# so there's 4141 metabolites in the model, ignoring compartmentalization
# Try to match name in Refmet

bad = []
for k in new_cpd_dict:
    x = new_cpd_dict[k][0].replace('(-)-', '').replace('(+)-', '').lower()
    if x not in lowrefmetDict:
        bad.append(x)
        
print(len(bad))

3594


In [44]:
bad[:22]

['(10z)-heptadecenoic acid',
 '(10z)-heptadecenoyl-coa',
 '(11r)-hpete',
 '(11z)-docosenoyl-coa',
 '(11z)-eicosenoyl-coa',
 '(11z,14z)-eicosadienoic acid',
 '(11z,14z)-eicosadienoyl-coa',
 '(11z,14z,17z)-eicosatrienoic acid',
 '(11z,14z,17z)-eicosatrienoylcarnitine',
 '(11z,14z,17z)-eicosatrienoyl-coa',
 '(13e)-11alpha-hydroxy-9,15-dioxoprost-13-enoate',
 '(13e)-tetranor-16-carboxy-lte4',
 '(13e)-tetranor-16-oxo-16-coa-lte4',
 '(13z)-docosenoyl-coa',
 '(13z)-eicosenoic acid',
 '(13z)-eicosenoyl-coa',
 '(13z)-octadecenoic acid',
 '(13z)-octadecenoyl-coa',
 '(13z,16z)-docosadienoic acid',
 '(13z,16z)-docosadienoylcarnitine',
 '(13z,16z)-docosadienoyl-coa',
 '(15z)-tetracosenoylcarnitine']

In [45]:
len(rxns)

13097

In [48]:
dir(rxns[0])

['__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'attrib',
 'clear',
 'extend',
 'find',
 'findall',
 'findtext',
 'get',
 'getchildren',
 'getiterator',
 'insert',
 'items',
 'iter',
 'iterfind',
 'itertext',
 'keys',
 'makeelement',
 'remove',
 'set',
 'tag',
 'tail',
 'text']

In [53]:
rxns[0].find('{http://www.sbml.org/sbml/level3/version1/core}annotation').findtext('metanetx')

In [56]:
t = open(xmlFile).read().splitlines()
print(len(t))
counter = 0
for L in t:
    if 'metanetx' in L:
        counter += 1
        
print(counter)

599525
7243


In [59]:
#
# Will check the JSON data in the repo
# shuzhao@canyon:~/projects/Azimuth/model_porting$ ls Human-GEM/data/annotation/
# humanGEMMetAssoc.JSON  humanGEMRxnAssoc.JSON

import json
jm = json.load(open('Human-GEM/data/annotation/humanGEMMetAssoc.JSON'))

print(jm.keys())

dict_keys(['mets', 'metsNoComp', 'metBiGGID', 'metKEGGID', 'metHMDBID', 'metChEBIID', 'metPubChemID', 'metLipidMapsID', 'metEHMNID', 'metHepatoNET1ID', 'metRecon3DID', 'metMetaNetXID'])


In [60]:
for k,v in jm.items(): 
    print(len(v))

10138
10138
10138
10138
10138
10138
10138
10138
10138
10138
10138
10138


In [61]:
for k,v in jm.items(): 
    print( v[:5] )

['m00001c', 'm00001s', 'm00002c', 'm00002s', 'm00003c']
['m00001', 'm00001', 'm00002', 'm00002', 'm00003']
['carveol', 'carveol', 'appnn', 'appnn', '']
['C00964', 'C00964', 'C09880', 'C09880', '']
['', '', 'HMDB06525', 'HMDB06525', '']
['CHEBI:15389', 'CHEBI:15389', 'CHEBI:36740', 'CHEBI:36740', '']
['', '', '6654', '6654', '']
['', '', '', '', 'LMFA01030283']
['', '', '', '', '']
['', '', '', '', '']
['carveol', 'carveol', 'appnn', 'appnn', 'M00003']
['MNXM45735', 'MNXM45735', 'MNXM163755', 'MNXM163755', 'MNXM150165; MNXM27815']


In [66]:
# so these appear to be flat lists of identifiers from other sources
# Maybe MetaNetX is more complete?

print(len(jm['metMetaNetXID']))
print(len(set(jm['metMetaNetXID'])))
print(len([x for x in jm['metMetaNetXID'] if x]))
print(len([x for x in jm['metRecon3DID'] if x]))
print(len([x for x in jm['metKEGGID'] if x]))


10138
2972
7326
10018
4599


In [65]:
# How many unique IDs in each?
for k,v in jm.items(): 
    print(k, len(set(v)))

mets 10138
metsNoComp 4168
metBiGGID 2347
metKEGGID 1647
metHMDBID 713
metChEBIID 1247
metPubChemID 1382
metLipidMapsID 487
metEHMNID 712
metHepatoNET1ID 773
metRecon3DID 5089
metMetaNetXID 2972


### Almost all metabolites have Recon3D ID

The Recon3D GitHub repo is too bulky to clone as a whole.

https://github.com/SBRG/Recon3D/tree/master/Recon3D_GP

Downloaded Recon3D_GP.json as individual file.

In [67]:
r3d = json.load(open('Recon3D/Recon3D_GP.json'))

print(r3d.keys())

dict_keys(['__instance_type__', 'attributes'])


In [71]:
r3d['__instance_type__']

['ssbio.pipeline.gempro', 'GEMPRO']

In [73]:
r3d['attributes'].keys()

dict_keys(['_root_dir', 'custom_spont_id', 'description', 'genome_path', 'id', 'model', 'notes', 'pdb_file_type'])

In [78]:
for k,v in r3d['attributes'].items():
    try:
        print(k, len(v))
    except TypeError:
        print(k, v)

_root_dir 1
custom_spont_id None
description None
genome_path None
id 10
model 2
notes 0
pdb_file_type 4


In [79]:
r3d['attributes']['id']

'Recon3D_GP'

In [80]:
r3d['attributes']['model'].keys()

dict_keys(['__instance_type__', 'attributes'])

In [81]:
r3d['attributes']['model']['attributes'].keys()

dict_keys(['reactions', 'metabolites', 'genes', 'id', 'name', 'compartments', 'version'])

In [82]:
len(r3d['attributes']['model']['attributes']['metabolites'])

8399

In [84]:
r3d['attributes']['model']['attributes']['metabolites'][200:202]

[{'id': '5htrp__91__c__93__',
  'name': '5-Hydroxy-L-Tryptophan',
  'compartment': 'c',
  'charge': 0,
  'formula': 'C11H12N2O3',
  'annotation': {'hmdb': 'HMDB00472',
   'inchi': 'InChI=1/C11H12N2O3/c12-9(11(15)16)3-6-5-13-10-2-1-7(14)4-8(6)10/h1-2,4-5,9,13-14H,3,12H2,(H,15,16)',
   'kegg.compound': 'C01017',
   'pubchem.compound': '144'}},
 {'id': 'srtn__91__c__93__',
  'name': 'Serotonin',
  'compartment': 'c',
  'charge': 1,
  'formula': 'C10H13N2O',
  'annotation': {'hmdb': 'HMDB00259',
   'inchi': 'InChI=1S/C10H12N2O/c11-4-3-7-6-12-10-2-1-8(13)5-9(7)10/h1-2,5-6,12-13H,3-4,11H2/p+1',
   'kegg.compound': 'C00780',
   'pubchem.compound': '5202'}}]

In [88]:
# How many r3d metabolites have pubchem.compound ID?

t = []
for x in r3d['attributes']['model']['attributes']['metabolites']:
    try:
        t.append(x['annotation']['pubchem.compound'])
    except KeyError:
        pass

print(len(t), len(set(t)))


3638 1368


In [89]:
#r3d['attributes']['model']['attributes']['metabolites'][3333]['annotation']['pubchem.compound']

def count_r3d(extID):
    t = []
    for x in r3d['attributes']['model']['attributes']['metabolites']:
        try:
            t.append(x['annotation'][extID])
        except KeyError:
            pass

    print(len(t), len(set(t)))

for x in ['hmdb', 'kegg.compound', 'pubchem.compound']:
    print(x, "---------------")
    count_r3d(x)    
    


hmdb ---------------
2170 699
kegg.compound ---------------
3592 1541
pubchem.compound ---------------
3638 1368


In [92]:
r3d_metabolites = [x['id'] for x in r3d['attributes']['model']['attributes']['metabolites']]
jmr3d = [x for x in jm['metRecon3DID'] if x]
jmr3d = list(set(jmr3d))

print(r3d_metabolites[:10])
print(jmr3d[:10])

cleanr3d = [x.split('__')[0] for x in r3d_metabolites]
cleanr3d = list(set(cleanr3d))
print(len(jmr3d), len(r3d_metabolites), len(cleanr3d))
print(cleanr3d[:10])

['10fthf5glu__91__c__93__', '10fthf5glu__91__l__93__', '10fthf5glu__91__m__93__', '10fthf6glu__91__c__93__', '10fthf6glu__91__l__93__', '10fthf6glu__91__m__93__', '10fthf7glu__91__c__93__', '10fthf7glu__91__l__93__', '10fthf7glu__91__m__93__', '10fthf__91__c__93__']
['35dsmv_e', '6ahglz_e', 'M00481', 'peamn', 'am19cs_e', 'acgalfucgalacgalfuc12gal14acglcgalgluside_hs', 'CE7110', '3octpencoa_m', 'C14849', 'CE2196']
5088 8399 4140
['CE5345', 'CE6508', 'emem2gacpail_hs', 'pmtcoa', 'serglyglu', 'cdpdag_hs', 'glumet', 'datp', 'ksi_pre7', 'CE2539']


In [93]:
# how many overlap in jmr3d and Recon3D?

len(set(jmr3d).intersection(cleanr3d))

3111

### But not sure how to link Recon3D ID to other DBs or information yet.

This will be updated at a later time to use Recon3D info.

See notes at the end of this notebook.

**compiling the mess**

10138
dict_keys(['mets', 'metsNoComp', 'metBiGGID', 'metKEGGID', 'metHMDBID', 'metChEBIID', 'metPubChemID', 'metLipidMapsID', 'metEHMNID', 'metHepatoNET1ID', 'metRecon3DID', 'metMetaNetXID'])


In [116]:
# from Azimuth_metabolomics
class Compound:
    def __init__(self):
        '''
        Azimuth ID starts with `az`, 
        and incorporates HMDB ID (less ambiguous than KEGG) whereas possible.
        Ions are precomputed as a dictionary.
        '''
        self.azimuth_id = ''
        self.name = ''          # common name
        self.other_ids = {
            'PubChem': '',
            'KEGG': '',
            'HMDB': '',
            'ChEBI': '',
            'MetaNetX': '',
            'metabolicATLAS': '',
            'recon3d': '',
            }
        self.formula = ''       # neutral or charged
        self.inchi = ''
        self.mw = 0             # mono_mass
        self.adducts = {}
        self.note = {
            'atlas2020': '',
        }

atlas2020metabolites = []

In [118]:
# use az_pcid_ for pubchem
# 
d = {}
def get_ii_jm(jm, ii):
    # wanted dict items from jm 
    return {
            'PubChem': jm['metPubChemID'][ii],
            'KEGG': jm['metKEGGID'][ii],
            'HMDB': jm['metHMDBID'][ii],
            'ChEBI': jm['metChEBIID'][ii],
            'MetaNetX': jm['metMetaNetXID'][ii],
            'metabolicATLAS': jm['metsNoComp'][ii],
            'recon3d': jm['metRecon3DID'][ii],
        }

def join_MNXM(d1, d2):
    # multiple entries for some cases for MetaNetX ID
    if d1['MetaNetX'] and d2['MetaNetX']:
        return "; ".join([d1['MetaNetX'], d2['MetaNetX']])
    else:
        return d1['MetaNetX'] + d2['MetaNetX']

for ii in range(10138):
    k,v = jm['metsNoComp'][ii], get_ii_jm(jm, ii)
    if k in d:
        if d[k] == v:
            pass
        else:
            print(d[k], v)
            d[k]['MetaNetX'] = join_MNXM(d[k], v)
    else:
        d[k] = v
     
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print(len(d))
print(d['m00003'])

{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM90540', 'metabolicATLAS': 'galgluside_hs', 'recon3d': 'galgluside_hs_c'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM90540', 'metabolicATLAS': 'galgluside_hs', 'recon3d': 'galgluside_hs_g'}
{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM4098', 'metabolicATLAS': 'Rtotal', 'recon3d': 'Rtotal_x'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM4098', 'metabolicATLAS': 'Rtotal', 'recon3d': 'Rtotal_c'}
{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM2279', 'metabolicATLAS': 'Rtotalcoa', 'recon3d': 'Rtotalcoa_c'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM2279', 'metabolicATLAS': 'Rtotalcoa', 'recon3d': 'Rtotalcoa_m'}
{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM4801', 'metabolicATLAS': 'Rtotal2coa', 'recon3d': 'Rtotal2coa_c'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', '

{'PubChem': '566787', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'c16dc', 'recon3d': 'c16dc_c'} {'PubChem': '566787', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'c16dc', 'recon3d': 'c16dc_e'}
{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM1653; MNXM1653; MNXM1653; MNXM1653', 'metabolicATLAS': 'hxa', 'recon3d': 'hxa_c'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': 'MNXM1653', 'metabolicATLAS': 'hxa', 'recon3d': 'hxa_e'}
{'PubChem': '6426853', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'c6crn', 'recon3d': 'c6crn_c'} {'PubChem': '6426853', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'c6crn', 'recon3d': 'c6crn_e'}
{'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'hexgly', 'recon3d': 'hexgly_c'} {'PubChem': '', 'KEGG': '', 'HMDB': '', 'ChEBI': '', 'MetaNetX': '', 'metabolicATLAS': 'hexgly', 'recon3d': 

In [119]:
# populate atlas2020metabolites list of Compounds

counter = 0

for k in d:
    C = Compound()
    if d[k]['PubChem']:
        C.azimuth_id = 'az_pcid_' + d[k]['PubChem']
    elif d[k]['HMDB']:
        C.azimuth_id = 'az_hmdb_' + d[k]['HMDB']
    elif d[k]['KEGG']:
        C.azimuth_id = 'az_kegg_' + d[k]['KEGG']
    elif d[k]['ChEBI']:
        C.azimuth_id = 'az_chebi_' + d[k]['ChEBI']
    else:
        C.azimuth_id = 'az_atlas_' + d[k]['metabolicATLAS']
        counter += 1
    
    C.other_ids = d[k]
    atlas2020metabolites.append(C)

print(len(atlas2020metabolites), counter)

4168 1975


**Among 4168 metabolites in the metabolic ATLAS model, 1975 have not been assigned ID from either PubChem, HMDB, KEGG or ChEBI.**



In [120]:
all = [C.azimuth_id for C in atlas2020metabolites]
print(len([x for x in all if 'az_pcid_' in x]))
print(len([x for x in all if 'az_hmdb_' in x]))
print(len([x for x in all if 'az_kegg_' in x]))
print(len([x for x in all if 'az_chebi_' in x]))

1419
15
734
25


In [121]:
# add info from Refmet 
# [' refmet_name', 'super_class', 'main_class', 'sub_class', 'formula', 'exactmass', 'inchi_key', 'pubchem_cid']

# index via pubchem_cid
pid_refmetDict = {}
for k,v in refmetDict.items():
    if v[7]:                    # this is 'pubchem_cid'
        pid_refmetDict[v[7]] = {
            'pubchem_cid': v[7],
            'refmet_name': v[0], 
            'super_class': v[1], 
            'main_class': v[2], 
            'sub_class': v[3], 
            'formula': v[4], 
            'exactmass': v[5], 
            'inchi_key': v[6],
        }

print("Done indexing - ", len(pid_refmetDict))

counter = 0

for C in atlas2020metabolites:
    if C.other_ids['PubChem'] and C.other_ids['PubChem'] in pid_refmetDict:
        counter += 1
        C.formula = pid_refmetDict[C.other_ids['PubChem']]['formula']
        C.mw = pid_refmetDict[C.other_ids['PubChem']]['exactmass']
        C.inchi_key = pid_refmetDict[C.other_ids['PubChem']]['inchi_key']
        C.refmet = pid_refmetDict[C.other_ids['PubChem']]

print(counter)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")


Done indexing -  23062
748
~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [122]:
print([C.formula for C in atlas2020metabolites[:30]])

['', '', '', '', '', '', '', '', '', '', '', '', 'C20H32O5', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'C20H30O3', 'C20H30O3', '']


**Only 748 ported from Refmet via PubChem ID**

Go thru mummichog 2 next, via KEGG like IDs.

This is via single file JSON_metabolicModels.py in v2.

In [123]:
from JSON_metabolicModels import metabolicModels

mcg2cpds = metabolicModels[ 'human_model_mfn' ]['Compounds']

print(list(mcg2cpds.items())[500:505])

[('G00123', {'formula': '', 'mw': 0, 'name': 'GA2; (Gal)1 (GalNAc)1 (Glc)1 (Cer)1; Glycolipid; Sphingolipid', 'adducts': {}}), ('G00025', {'formula': '', 'mw': 0, 'name': '(Gal)1 (GalNAc)1 (GlcNAc)1 (Ser/Thr)1; Glycoprotein; O-Glycan', 'adducts': {}}), ('C04833', {'formula': '', 'mw': 0, 'name': "m7G(5')pppm6Am (mRNA containing an N6,2'-O-dimethyladenosine cap)", 'adducts': {}}), ('dcsptn1coa', {'formula': 'C43H64N7O17P3S', 'mw': 1075.329224, 'name': 'docosa-4,7,10,13,16-pentaenoyl coenzyme A', 'adducts': {'M+2H[2+]': 538.67188846677, 'M+Br81[-]': 1156.2455240000002, 'M-H2O+H[1+]': 1058.32590046677, 'M-C3H4O2+H[1+]': 1004.31540046677, 'M-HCOOH+H[1+]': 1030.33110046677, 'M-CO+H[1+]': 1048.3415004667702, 'M+K[1+]': 1114.29200046677, 'M+Cl[-]': 1110.2981240000001, 'M+Na-2H[-]': 1095.29667106646, 'M-CO2+H[1+]': 1032.34670046677, 'M+Na[1+]': 1098.31850046677, 'M-2H[2-]': 536.65733553323, 'M(S34)-H[-]': 1076.31774753323, 'M+H[1+]': 1076.33650046677, 'M-H4O2+H[1+]': 1040.3153004667702, 'M(C13

In [124]:
counter = 0

for C in atlas2020metabolites:
    if C.other_ids['KEGG'] and C.other_ids['KEGG'] in mcg2cpds:
        counter += 1
        C.formula = mcg2cpds[C.other_ids['KEGG']]['formula']
        C.mw = mcg2cpds[C.other_ids['KEGG']]['mw']
        C.mummichog2 = mcg2cpds[C.other_ids['KEGG']]

print(counter)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")

1462
~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [125]:
print(len([C.formula for C in atlas2020metabolites if C.formula]))
print(len([C.mw for C in atlas2020metabolites if C.mw]))


1264
1334


This may have overwritten some entries from Refmet.

mcg2 adducts need to be calculated again:
* to fix the bug Jasmine found on -ion Na/K
* not to be blanket when formula is missing

Another round via Refmet by names:

In [127]:
name_refmetDict = {}
for k,v in lowrefmetDict.items():
    name_refmetDict[k] = {
            'pubchem_cid': v[7],
            'refmet_name': v[0], 
            'super_class': v[1], 
            'main_class': v[2], 
            'sub_class': v[3], 
            'formula': v[4], 
            'exactmass': v[5], 
            'inchi_key': v[6],
        }

print("Done indexing - ", len(name_refmetDict))

# also need build dict from C.other_ids['metabolicATLAS'] to common name
atlas_names = {}
for k,v in cpd_dict.items():
    atlas_names[k[2:-1]] = v['name']  # changing key, e.g. 'M_m00001c' to 'm00001'

#
counter = 0
unfound = []
for C in atlas2020metabolites:
    if not C.mw:
        # try get name via C.other_ids['metabolicATLAS']
        try:
            lname = atlas_names[C.other_ids['metabolicATLAS']].replace('(-)-', '').replace('(+)-', '').lower()
            if lname in name_refmetDict:
                counter += 1
                C.formula = name_refmetDict[lname]['formula']
                C.mw = name_refmetDict[lname]['exactmass']
                C.inchi_key = name_refmetDict[lname]['inchi_key']
                C.refmet = name_refmetDict[lname]
        except KeyError:
            unfound.append(C.other_ids['metabolicATLAS'])
            
            
print(counter)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print (len(unfound), unfound[:60])

Done indexing -  95384
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~
914 ['temp002', 'temp003', 'temp004', 'temp005', '3amp', 'galgluside_hs', 'Rtotal2', 'Rtotal2coa', 'alpa_hs', 'alkylR1oh', 'Rtotal', 'glygn4', 'Rtotalcoa', 'Rtotal3coa', 'Rtotal3', 'R1coa_hs', 'R2coa_hs', 'R3coa_hs', 'octd11ecoa', 'R4coa_hs', 'R5coa_hs', 'R6coa_hs', 'galacglcgalacglcgal14acglcgalgluside_hs', 'dgcholcoa', 'bvite', '2dpmhobq', '2dp6mobq', '2dp6mobq_me', 'dmnoncoa', 'melanin', 'Ser_Thr', 'dmhptcoa', 'glc1man', 'glc2man', 'glc3man', 'etfox', 'etfrd', '1glyc_hs', '9_cis_retfa', 'fucfuc12gal14acglcgalgluside_hs', 'fucfucgalacglcgalgluside_hs', 'glyc_S', 'glygn5', 'Tyr_ggn', 'xolest_hs', 'tdeACP', 'hdeACP', 'ocdcaACP', 'octeACP', 'lnlcACP', 'lneldcACP', 'ocdcyaACP', 'ocdcya', 'pristcoa', 'tag1p_D', 'm_em_3gacpail', 'lnlncacrn', 'm_em_3gacpail_hs', 'm_em_3gacpail_prot_hs', 'pecgoncoa']


**This means most names in ATLAS XML are not names but BiGG style IDs**

### Looking for all metabolites in ATLAS model via vmh.life API

Recon3D site's API: https://www.vmh.life/_api/docs/#metabolites-list


In [114]:
import requests

vmh_results = []
counter = 0
for C in atlas2020metabolites:
    counter += 1
    url = "https://www.vmh.life/_api/metabolites/?abbreviation=%s&format=json" %C.other_ids['metabolicATLAS']
    r = requests.get(url).json()
    vmh_results += r['results']
    if str(counter)[-2:] == '00':
        print(counter)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100


In [115]:
# above API calls took > 1 hour
len(vmh_results)

1787

In [134]:
# check how many unfound in vmh_results

vabbr = [x['abbreviation'].lower() for x in vmh_results]
print(len(vabbr), len(set(vabbr).intersection(set(unfound))))

print([x for x in vabbr if x not in unfound][:30])
print(list(set(vabbr).intersection(set(unfound)))[:30])

1787 784
['m00003', 'm00004', 'm00005', 'm00006', 'm00008', 'm00010', 'm00011', 'm00012', 'm00017', 'm00018', 'm00019', 'm00020', 'm00021', 'm00022', 'm00023', 'm00030', 'm00031', 'm00033', 'm00034', 'm00044', 'm00045', 'm00046', 'm00047', 'm00048', 'm00049', 'm00054', 'm00055', 'm00056', 'm00060', 'm00061']
['tlacfvs', 'crglz', 'micit', 'lineth', 'histrphis', 'serglyglu', '1hmdgluc', 'glumet', 'am19cs', '3ohsubcoa', 'tmdm5', '3hdececrn', 'alaglylys', '4hatvacid', 'pmeth', 'c16dc', 'mdz', 'argphearg', 'asnpheasp', 'ileasnhis', 'valtrpval', 'hexde7coa', 'pcholpalm_hs', 'argprothr', 'trpglupro', 'hislysthr', '2hiv', 'prophe', 'lnlncacrn', 'sphmyln181161_hs']


In [132]:
list(cpd_dict.items())[:2]

[('M_m00001c',
  {'metaid': 'M_m00001c',
   'sboTerm': 'SBO:0000247',
   'id': 'M_m00001c',
   'name': '(-)-trans-carveol',
   'compartment': 'c',
   'initialConcentration': '0',
   'hasOnlySubstanceUnits': 'false',
   'boundaryCondition': 'false',
   'constant': 'false',
   '{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge': '0',
   '{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula': 'C10H16O'}),
 ('M_m00001s',
  {'metaid': 'M_m00001s',
   'sboTerm': 'SBO:0000247',
   'id': 'M_m00001s',
   'name': '(-)-trans-carveol',
   'compartment': 's',
   'initialConcentration': '0',
   'hasOnlySubstanceUnits': 'false',
   'boundaryCondition': 'false',
   'constant': 'false',
   '{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge': '0',
   '{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula': 'C10H16O'})]

In [137]:
# update d with charge and formula

for k,v in cpd_dict.items():
    try:
        d[k[2:-1]]['charge'] = int(v['{http://www.sbml.org/sbml/level3/version1/fbc/version2}charge'])
    except KeyError:
        pass
    try:
        d[k[2:-1]]['formula'] = v['{http://www.sbml.org/sbml/level3/version1/fbc/version2}chemicalFormula']
    except KeyError:
        pass

list(d.items())[2]

('m00003',
 {'PubChem': '',
  'KEGG': '',
  'HMDB': '',
  'ChEBI': '',
  'MetaNetX': 'MNXM150165; MNXM27815',
  'metabolicATLAS': 'm00003',
  'recon3d': 'M00003',
  'charge': -1,
  'formula': 'C17H31O2'})

In [140]:
good = [C.mw for C in atlas2020metabolites if C.mw]
print(len(good))

counter = 0
for C in atlas2020metabolites:
    ii = C.other_ids['metabolicATLAS']
    if 'charge' in d[ ii ]:
        C.charge = d[ii]['charge']
        counter += 1
    if 'formula' in d[ii]:
        C.formula = d[ii]['formula']

print(counter)
print(len([C.formula for C in atlas2020metabolites if C.formula]))

1394
3196
3249


In [141]:
vmh_results[55]

{'met_id': 5545,
 'abbreviation': 'M00187',
 'createdDate': '2018-05-31T15:54:08.170242Z',
 'updatedDate': '2018-08-10T14:18:12.616036Z',
 'fullName': 'Beta-Adrenergic-Receptor',
 'description': '',
 'synonyms': None,
 'iupac': None,
 'neutralFormula': '',
 'chargedFormula': 'C4H6N2O3R2',
 'charge': 0,
 'avgmolweight': None,
 'monoisotopicweight': None,
 'miriam': 'http://identifiers.org/vmhmetabolite/M00187',
 'biggId': '',
 'lmId': '',
 'ehmnId': '',
 'hepatonetId': '',
 'keggId': 'C01141',
 'pubChemId': '',
 'cheBlId': '',
 'chembl': '',
 'inchiString': '',
 'inchiKey': None,
 'smile': '[H]OC([H])([H])[C@]([H])(N([H])C([*])=O)C(=O)N([H])[*]',
 'hmdb': '',
 'metanetx': '',
 'seed': None,
 'pdmapName': None,
 'reconMap': None,
 'reconMap3': None,
 'golgimap': None,
 'lysosomemap': None,
 'mitochondrionmap': None,
 'nucleusmap': None,
 'peroxisomemap': None,
 'reticulummap': None,
 'food_db': None,
 'chemspider': None,
 'biocyc': None,
 'wikipedia': None,
 'drugbank': None,
 'knapsack'

**The above result shows an nonsense idiosyncrasy started in KEGG but no one fixed it**

How could a GPCR protein be treated as a "metabolite" for ever???

In [147]:
vmh_results[1600]

{'met_id': 5708,
 'abbreviation': 'lca24g',
 'createdDate': '2018-05-31T15:54:08.170242Z',
 'updatedDate': '2018-12-03T15:53:11.425813Z',
 'fullName': 'Lithocholic acid-24glucuronide, CDCA-24G',
 'description': '',
 'synonyms': '',
 'iupac': None,
 'neutralFormula': '',
 'chargedFormula': 'C30H47O9',
 'charge': -1,
 'avgmolweight': None,
 'monoisotopicweight': None,
 'miriam': 'http://identifiers.org/vmhmetabolite/lca24g',
 'biggId': '',
 'lmId': '',
 'ehmnId': '',
 'hepatonetId': '',
 'keggId': '',
 'pubChemId': '',
 'cheBlId': '',
 'chembl': '',
 'inchiString': '',
 'inchiKey': None,
 'smile': '',
 'hmdb': '',
 'metanetx': '',
 'seed': None,
 'pdmapName': '',
 'reconMap': '',
 'reconMap3': 'lca24g',
 'golgimap': None,
 'lysosomemap': None,
 'mitochondrionmap': None,
 'nucleusmap': None,
 'peroxisomemap': None,
 'reticulummap': 'lca24g',
 'food_db': None,
 'chemspider': None,
 'biocyc': None,
 'wikipedia': None,
 'drugbank': None,
 'knapsack': None,
 'phenolExplorer': None,
 'metlin':

In [143]:
# How many we might gain from vmh_results

unfound = [C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]
print(len(vabbr), len(set(vabbr).intersection(set(unfound))))

1787 786


In [150]:
#
vabbrDict = {}
for x in vmh_results:
    vabbrDict[x['abbreviation'].lower()] = x
    
for C in atlas2020metabolites:
    k = C.other_ids['metabolicATLAS']
    if not C.formula and k in vabbrDict:
        if vabbrDict[k]['charge']:
            C.charge = vabbrDict[k]['charge']
            C.formula = vabbrDict[k]['chargedFormula']
        elif vabbrDict[k]['neutralFormula']:
            C.formula = vabbrDict[k]['neutralFormula']
        C.vmh = vabbrDict[k]

print(len([C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]))
print(len([C for C in atlas2020metabolites if C.formula]))
print(len(atlas2020metabolites))

412
3756
4168


In [157]:
remaining = [C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]
print(remaining)

['temp002', 'temp003', 'temp004', 'temp005', 'galgluside_hs', 'Rtotal2', 'Rtotal2coa', 'alkylR1oh', 'Rtotal', 'glygn4', 'Rtotalcoa', 'Rtotal3coa', 'Rtotal3', 'R1coa_hs', 'R2coa_hs', 'R3coa_hs', 'R4coa_hs', 'R5coa_hs', 'R6coa_hs', 'galacglcgalacglcgal14acglcgalgluside_hs', 'bvite', '2dpmhobq', '2dp6mobq', '2dp6mobq_me', 'Ser_Thr', 'glc1man', 'glc2man', 'glc3man', 'etfox', 'etfrd', '9_cis_retfa', 'fucfuc12gal14acglcgalgluside_hs', 'fucfucgalacglcgalgluside_hs', 'glyc_S', 'glygn5', 'Tyr_ggn', 'xolest_hs', 'tdeACP', 'hdeACP', 'ocdcaACP', 'octeACP', 'lnlcACP', 'lneldcACP', 'ocdcyaACP', 'tag1p_D', 'm_em_3gacpail', 'lnlncacrn', 'm_em_3gacpail_hs', 'm_em_3gacpail_prot_hs', 'Rtotal2crn', 'Rtotal3crn', 'Rtotalcrn', 'HC02119', 'HC02097', 'HC02098', 'HC02099', 'HC01988', 'HC02111', 'HC02112', 'HC02113', 'HC02114', 'HC02115', 'CE0692', 'HC02154', 'CE2881', 'CE2885', 'CE2882', 'CE2886', 'CE2883', 'CE2887', 'CE2884', 'CE2888', 'malthx', 'malthp', 'CE1950', 'CE2891', 'CE4753', 'CE4754', 'CE0233', 'CE2

In [154]:
# rerun vmh API, some may be timed out last time
counter = 0
for x in remaining:
    counter += 1
    url = "https://www.vmh.life/_api/metabolites/?abbreviation=%s&format=json" %x
    r = requests.get(url).json()
    vmh_results += r['results']
    if str(counter)[-2:] == '00':
        print(counter)

100
200
300
400


In [158]:
len(vmh_results)
vmh_results[1787 + 20]

{'met_id': 725,
 'abbreviation': 'Ser_Thr',
 'createdDate': '2018-05-31T15:54:08.170242Z',
 'updatedDate': '2019-01-29T17:58:36.589634Z',
 'fullName': 'Protein-Linked Serine Or Threonine Residue (O-Glycosylation Site)',
 'description': '',
 'synonyms': None,
 'iupac': None,
 'neutralFormula': '',
 'chargedFormula': 'XH',
 'charge': 0,
 'avgmolweight': None,
 'monoisotopicweight': None,
 'miriam': 'http://identifiers.org/vmhmetabolite/Ser_Thr',
 'biggId': 'Ser_Thr',
 'lmId': '',
 'ehmnId': '',
 'hepatonetId': '',
 'keggId': '',
 'pubChemId': '',
 'cheBlId': '',
 'chembl': '',
 'inchiString': '',
 'inchiKey': None,
 'smile': '',
 'hmdb': '',
 'metanetx': 'MNXM147296',
 'seed': None,
 'pdmapName': None,
 'reconMap': 'Ser_Thr',
 'reconMap3': 'Ser_Thr',
 'golgimap': 'Ser_Thr',
 'lysosomemap': 'Ser_Thr',
 'mitochondrionmap': None,
 'nucleusmap': None,
 'peroxisomemap': None,
 'reticulummap': None,
 'food_db': None,
 'chemspider': None,
 'biocyc': None,
 'wikipedia': None,
 'drugbank': None,


In [159]:
# paste & rerun the prevous block

vabbrDict = {}
for x in vmh_results:
    vabbrDict[x['abbreviation'].lower()] = x
    
for C in atlas2020metabolites:
    k = C.other_ids['metabolicATLAS']
    if not C.formula and k in vabbrDict:
        try:
            C.charge = vabbrDict[k]['charge']
            C.formula = vabbrDict[k]['chargedFormula']
        except KeyError:
            try:
                C.formula = vabbrDict[k]['neutralFormula']
            except KeyError:
                print(k)
        C.vmh = vabbrDict[k]

print(len([C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]))
print(len([C for C in atlas2020metabolites if C.formula]))
print(len(atlas2020metabolites))

133
4035
4168


In [160]:
remaining = [C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]
print(len(remaining))
print(remaining)

133
['temp002', 'temp003', 'temp004', 'temp005', 'Rtotal2', 'Rtotal2coa', 'alkylR1oh', 'Rtotal', 'Rtotalcoa', 'Rtotal3coa', 'Rtotal3', 'R1coa_hs', 'R2coa_hs', 'R3coa_hs', 'R4coa_hs', 'R5coa_hs', 'R6coa_hs', 'Ser_Thr', 'glyc_S', 'Tyr_ggn', 'tdeACP', 'hdeACP', 'ocdcaACP', 'octeACP', 'lnlcACP', 'lneldcACP', 'ocdcyaACP', 'tag1p_D', 'Rtotal2crn', 'Rtotal3crn', 'Rtotalcrn', 'HC02119', 'HC02097', 'HC02098', 'HC02099', 'HC01988', 'HC02111', 'HC02112', 'HC02113', 'HC02114', 'HC02115', 'CE0692', 'HC02154', 'CE2881', 'CE2885', 'CE2882', 'CE2886', 'CE2883', 'CE2887', 'CE2884', 'CE2888', 'CE1950', 'CE2891', 'CE4753', 'CE4754', 'CE0233', 'CE2953', 'CE1162', 'CE2955', 'CE5775', 'CE5776', 'CE1294', 'CE5932', 'CE5013', 'CE5837', 'CE5838', 'CE5839', 'CE5840', 'CE5829', 'CE5865', 'CE5866', 'CE5016', 'CE2615', 'CE2616', 'CE2597', 'CE2596', 'CE5117', 'CE5118', 'CE5119', 'CE5120', 'CE4790', 'CE4792', 'CE4794', 'CE4824', 'CE2313', 'CE6027', 'CE4837', 'CE5049', 'CE5160', 'CE5161', 'CE5162', 'CE5168', 'CN0012'

**So we are getting formula for 4035 of 4168 compounds.**

Exact mass can be computed from formula.
Pay attention to
neutral vs charged formula and mass!!

In [162]:
# mummichog has a few more
print(len(mcg2cpds),
mcg2cpds['CE2881'])

3557 {'formula': 'C14H7I4O4', 'mw': 746.6523057131, 'name': "3,5,3',5'-tetraiodothyroacetate", 'adducts': {'M+2H[2+]': 374.33342932332, 'M+Br81[-]': 827.5686057130999, 'M-H2O+H[1+]': 729.64898217987, 'M-C3H4O2+H[1+]': 675.6384821798699, 'M-HCOOH+H[1+]': 701.65418217987, 'M-CO+H[1+]': 719.66458217987, 'M+K[1+]': 785.61508217987, 'M+Cl[-]': 781.6212057130999, 'M+Na-2H[-]': 766.6197527795599, 'M-CO2+H[1+]': 703.66978217987, 'M+Na[1+]': 769.6415821798699, 'M-2H[2-]': 372.31887638978, 'M+H[1+]': 747.65958217987, 'M-H4O2+H[1+]': 711.63838217987, 'M(C13)-H[-]': 746.64842924633, 'M+HCOONa[1+]': 815.64698217987, 'M(C13)+2H[2+]': 374.83512932332, 'M+HCOOK[1+]': 831.62088217987, 'M+HCOO[-]': 791.6499507131, 'M(C13)+3H[3+]': 250.22577837113664, 'M+ACN-H[-]': 786.6715742463299, 'M+Cl37[-]': 783.6182057131, 'M-H2O-H[-]': 727.63442924633, 'M+Br[-]': 825.5706057131, 'M+3H[3+]': 249.89137837113665, 'M+CH3COO[-]': 805.6656007130999, 'M(C13)+H[1+]': 748.66298217987, 'M[1+]': 746.6523057131, 'M-H[-]': 745

In [163]:
for C in atlas2020metabolites:
    k = C.other_ids['metabolicATLAS']
    if not C.formula and k in mcg2cpds:
        C.formula = mcg2cpds[k]['formula']
        C.mw = mcg2cpds[k]['mw']
        C.mummichog2 = mcg2cpds[k]

print(len([C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]))
print(len([C for C in atlas2020metabolites if C.formula]))
print(len(atlas2020metabolites))

75
4093
4168


In [164]:
remaining = [C.other_ids['metabolicATLAS'] for C in atlas2020metabolites if not C.formula]
print(len(remaining))
print(remaining)

75
['temp002', 'temp003', 'temp004', 'temp005', 'alkylR1oh', 'Rtotal3', 'R1coa_hs', 'R2coa_hs', 'R3coa_hs', 'R4coa_hs', 'R5coa_hs', 'R6coa_hs', 'Ser_Thr', 'glyc_S', 'Tyr_ggn', 'tag1p_D', 'HC02119', 'HC02097', 'HC02098', 'HC02099', 'HC01988', 'HC02111', 'HC02112', 'HC02113', 'HC02114', 'HC02115', 'HC02154', 'CE0233', 'CE1162', 'CE2955', 'CE1294', 'CE5829', 'CE5865', 'CE5866', 'CE5160', 'CN0012', 'CN0009', 'CN0010', 'CN0011', 'CN0021', 'CN0022', 'CN0023', 'CN0017', 'CN0018', 'CN0019', 'abt_D', 'acthr_L', 'acile_L', 'acleu_L', 'achom_L', 'Lhcystin', '2hxic_L', '1hibupglu_S', '1hibup_S', '2hibupglu_S', '2hibup_R', '2hibup_S', '3hibupglu_S', '3hibup_R', '3hibup_S', 'caribupglu_S', 'caribup_R', 'ibup_R', 'ibup_S', 'tauribup_S', 'ibupcoa_R', 'ibupcoa_S', 'm10000', 'm10001', 'm10002', 'm10003', 'm10012', 'm10013', 'm10014', 'm10015']


## Finished collecting formula on metabolites
Next:
* Recompute mw and adducts


In [176]:
print(len([C for C in atlas2020metabolites if C.formula]))

withCharge = []
for C in atlas2020metabolites:
    try:
        withCharge.append((C.charge, C.formula, C.mw, C))
    except AttributeError:
        pass
        
print(len(withCharge))
print(len([x for x in withCharge if x != 0]))
print(withCharge[:5])
withCharge = [C[-1].azimuth_id for C in withCharge]

4093
3897
3897
[(0, 'C10H16O', 152.1201, <__main__.Compound object at 0x7f93ffc1e6d0>), (0, 'C10H16', '136.1252', <__main__.Compound object at 0x7f93ffc1e7d0>), (-1, 'C17H31O2', 0, <__main__.Compound object at 0x7f93ffc1e990>), (-4, 'C38H62N7O17P3S', 0, <__main__.Compound object at 0x7f93ffc1e510>), (-1, 'C20H31O4', 0, <__main__.Compound object at 0x7f93ffc23c50>)]


In [179]:
# verify charged formula against mcg 2

withKEGG = []
for C in atlas2020metabolites:
    if C.other_ids["KEGG"]:
        withKEGG.append(C)

new, good, bad = [], [], []
for C in withKEGG:
    if C.azimuth_id in withCharge and C.other_ids["KEGG"] in mcg2cpds:
        new.append(C)

print(len(new))
for C in new:
    if C.formula and C.formula == mcg2cpds[C.other_ids["KEGG"]]['formula']:
        good.append(C)
    else:
        bad.append(C)
        
print(len(good), len(bad))
for C in bad[200:220]:
    print(C.charge, C.formula, "||||||||", mcg2cpds[C.other_ids["KEGG"]]['formula'])

1449
433 1016
-2 C6H6O5 |||||||| C6H8O5
-1 C4H5O3 |||||||| C4H6O3
-1 C5H6NO4 |||||||| C5H7NO4
-3 C3H4O7P |||||||| C3H7O7P
-3 C2H2O6P |||||||| C2H5O6P
-1 C12H18N3O8S |||||||| 
0 C54H89N2O25P2R7 |||||||| 
-1 C8H7O5 |||||||| C8H8O5
-1 C8H7O4 |||||||| C8H8O4
-2 C66H111N4O38RCO |||||||| 
-4 C48H74N7O21P3S |||||||| C48H78N7O21P3S
-4 C48H74N7O20P3S |||||||| C48H78N7O20P3S
-1 C27H45O5 |||||||| C27H46O5
-4 C48H74N7O20P3S |||||||| C48H78N7O20P3S
-4 C48H74N7O19P3S |||||||| C48H78N7O19P3S
-1 C27H45O4 |||||||| 
1 C3H8NO |||||||| C3H7NO
-1 C23H36O20X |||||||| 
-3 C16H21N4O10P2S |||||||| C16H25N4O10P2S
-1 C6H9O7 |||||||| C6H10O7


**How many are caused by H atoms alone?**

In [184]:
def compareFormulas(x, y):
    p = []
    n1, n2 = len(x), len(y)
    if n1 == n2:
        for ii in range(n1):
            if x[-ii] != y[-ii]:
                p += [x[-ii-1], x[-ii]]

        return ''.join(p)
    else:
        return '~~~~'.join([x,y])

diffBad = []
for C in bad:
    y = mcg2cpds[C.other_ids["KEGG"]]['formula']
    if y:
        diffBad.append( [C.charge, C.formula, y, "^^^^^^^^^^^^^^^^^", compareFormulas(C.formula, y)] )
        
diffBad[30:50]

[[-4, 'C35H58N7O18P3S', 'C35H62N7O18P3S', '^^^^^^^^^^^^^^^^^', '58H5'],
 [-1, 'C5H5N2O4', 'C5H6N2O4', '^^^^^^^^^^^^^^^^^', 'H5'],
 [-4, 'C31H50N7O18P3S', 'C31H54N7O18P3S', '^^^^^^^^^^^^^^^^^', '50'],
 [-4, 'C27H42N7O18P3S', 'C27H46N7O18P3S', '^^^^^^^^^^^^^^^^^', '42'],
 [-4, 'C29H46N7O18P3S', 'C29H50N7O18P3S', '^^^^^^^^^^^^^^^^^', '46H4'],
 [1,
  'C9H18N3O2R2',
  'C8H18N2O2',
  '^^^^^^^^^^^^^^^^^',
  'C9H18N3O2R2~~~~C8H18N2O2'],
 [-2, 'C4H5N2O6PR2', 'HO4P', '^^^^^^^^^^^^^^^^^', 'C4H5N2O6PR2~~~~HO4P'],
 [0, 'C5H10O6PR', 'C19H39O6P', '^^^^^^^^^^^^^^^^^', 'PR6PO60O10H15HC5'],
 [-4, 'C3H4O10P2', 'C3H8O10P2', '^^^^^^^^^^^^^^^^^', 'H4'],
 [2, 'C3H12N2', 'C3H10N2', '^^^^^^^^^^^^^^^^^', '12'],
 [-2, 'C20H21N7O7', 'C20H23N7O7', '^^^^^^^^^^^^^^^^^', '21'],
 [-1, 'C20H31O3', 'C20H32O3', '^^^^^^^^^^^^^^^^^', '31'],
 [0, 'C21H30O3', 'C20H28O4', '^^^^^^^^^^^^^^^^^', 'O330H321'],
 [-1, 'C18H31O3', 'C18H32O3', '^^^^^^^^^^^^^^^^^', '31'],
 [-1, 'C20H31O4', 'C20H32O4', '^^^^^^^^^^^^^^^^^', '31'],
 [-1, 

In [191]:
print("Mummichog v2 has %s cpds" %len(mcg2cpds))

t = []
for k,v in mcg2cpds.items():
    if v['formula']:
        t.append(k)
        
print(len(t))
# How mnay R in model?
print("Number of metabolites in this model w/ R ", 
len([C for C in atlas2020metabolites if C.formula and "R" in C.formula]))

print(len(atlas2020metabolites))

Mummichog v2 has 3557 cpds
2170
Number of metabolites in this model w/ R  516
4168


**So the charge is mostly on H atoms**

**But the "R" group is nopass for mw, not usable anyway**

**Excluding the ones wiht R group, this model doesn't have many more metabolites than mcg v2. Hoping the formulas help**


In [198]:
# new mass calculation
# using pychemy module. It failed to install using pip because of `Open Babel` binding.
# But the mass calcuation function is not depdendent on `Open Babel`, 
# thus pychemy folder is copied here and it's usable as below

from pychemy.molmass import Formula

t = []

for C in atlas2020metabolites:
    if C.formula:
        if "R" not in C.formula and "X" not in C.formula:
            mw = Formula( C.formula ).isotope.mass
            try:
                t.append((mw, C.mw, C.formula, C.charge))
            except AttributeError:
                t.append((mw, C.mw, C.formula, ''))
                
print(len(t))
t[30:130]

3366


[(449.1256707986, 451.1413, 'C20H23N3O7S', -2),
 (144.0575148789, 144.0575, 'C10H8O', 0),
 (449.1256707986, 451.1413, 'C20H23N3O7S', -2),
 (144.0575148789, 144.0575, 'C10H8O', 0),
 (416.3290452787, 0, 'C27H44O3', 0),
 (1211.4027827601, 1215.4341, 'C48H76N7O21P3S', -4),
 (1195.4078681380001, 1199.4392, 'C48H76N7O20P3S', -4),
 (915.2040233729, 919.2353, 'C31H48N7O17P3S', -4),
 (1083.3918241433, 0, 'C43H72N7O17P3S', -4),
 (943.2353235013, 947.2666, 'C33H52N7O17P3S', -4),
 (1055.3605240149, 0, 'C41H68N7O17P3S', -4),
 (1069.3761740791, 0, 'C42H70N7O17P3S', -4),
 (1013.3135738223, 0, 'C38H62N7O17P3S', -4),
 (873.1570731803, 0, 'C28H42N7O17P3S', -4),
 (1139.4544244001, 0, 'C47H80N7O17P3S', -4),
 (999.2979237581, 1003.3292, 'C37H60N7O17P3S', -4),
 (859.1414231161, 863.1727, 'C27H40N7O17P3S', -4),
 (1041.3448739507, 0, 'C40H66N7O17P3S', -4),
 (901.1883733087, 0, 'C30H46N7O17P3S', -4),
 (1027.3292238865001, 1031.3605, 'C39H64N7O17P3S', -4),
 (887.1727232445, 891.204, 'C29H44N7O17P3S', -4),
 (985

**From visual inspection, the charge is responsible for diff mass there**

Continue calculating all mw using formula via the pychemy.molmass.Formula

In [199]:
print(len([C for C in atlas2020metabolites if C.mw]))

for C in atlas2020metabolites:
    if C.formula:
        if "R" not in C.formula and "X" not in C.formula:
            mw = Formula( C.formula ).isotope.mass
            C.mw = mw
            
print(len([C for C in atlas2020metabolites if C.mw]))

1437
3417


**Yet to compute adducts**

But need upload to GCP Azimuth now.

## Adding pathways to rxns


In [202]:
print(len(rxns))

rxn_dict = {}
for r in rxns:
    if r.attrib:
        tmp = {'id': r.attrib['id'], 'name': '', 'source': 'Human-GEM', }
        tmp['reactants'] = [x.attrib['species'] for x in r.find('{http://www.sbml.org/sbml/level3/version1/core}listOfReactants')]
        tmp['products'] = [x.attrib['species'] for x in r.find('{http://www.sbml.org/sbml/level3/version1/core}listOfProducts')]
        tmp['cpds'] = tmp['reactants'] + tmp['products']
        rxn_dict[r.attrib['id']] = tmp
        
print("Got %d reactions." %len(rxn_dict))
print(list(rxn_dict.items())[:2])

# will add pathway etc. from spreadsheet

13097
Got 13097 reactions.
[('R_HMR_3905', {'id': 'R_HMR_3905', 'name': '', 'source': 'Human-GEM', 'reactants': ['M_m01796c', 'M_m02552c'], 'products': ['M_m01249c', 'M_m02039c', 'M_m02553c'], 'cpds': ['M_m01796c', 'M_m02552c', 'M_m01249c', 'M_m02039c', 'M_m02553c']}), ('R_HMR_3907', {'id': 'R_HMR_3907', 'name': '', 'source': 'Human-GEM', 'reactants': ['M_m01796c', 'M_m02554c'], 'products': ['M_m01249c', 'M_m02039c', 'M_m02555c'], 'cpds': ['M_m01796c', 'M_m02554c', 'M_m01249c', 'M_m02039c', 'M_m02555c']})]


In [206]:
print(header)
print("EC-NUMBER", header.index('EC-NUMBER'))
print("SUBSYSTEM", header.index('SUBSYSTEM'))

print(sheets[0].row_values(2))

['#', 'ID', 'NAME', 'EQUATION', 'EC-NUMBER', 'GENE ASSOCIATION', 'LOWER BOUND', 'UPPER BOUND', 'OBJECTIVE', 'COMPARTMENT', 'MIRIAM', 'SUBSYSTEM', 'REPLACEMENT ID', 'NOTE', 'REFERENCE', 'CONFIDENCE SCORE']
EC-NUMBER 4
SUBSYSTEM 11
['', 'HMR_3907', '', 'ethanol[c] + NADP+[c] => acetaldehyde[c] + H+[c] + NADPH[c]', 'EC:1.1.1.2', 'ENSG00000117448', '', '', '', '', '', 'Glycolysis / Gluconeogenesis', '', '', '', 0.0]


In [208]:
len(rxns) 
# 13417 rows in RXNS sheet, 147 pathways

bad = []
for ii in range(1, sheets[0].nrows):
    thisdata = sheets[0].row_values(ii)
    rid, ec, pathway = thisdata[1], thisdata[4], thisdata[11]
    try:
        rxn_dict['R_' + rid]['EC'] = ec
        rxn_dict['R_' + rid]['pathway'] = pathway
    except KeyError:
        bad.append(thisdata[:5])
        
print(len(bad))

1515


In [209]:
print(list(rxn_dict.items())[:2])

[('R_HMR_3905', {'id': 'R_HMR_3905', 'name': '', 'source': 'Human-GEM', 'reactants': ['M_m01796c', 'M_m02552c'], 'products': ['M_m01249c', 'M_m02039c', 'M_m02553c'], 'cpds': ['M_m01796c', 'M_m02552c', 'M_m01249c', 'M_m02039c', 'M_m02553c'], 'EC': 'EC:1.1.1.1; EC:1.1.1.71', 'pathway': 'Glycolysis / Gluconeogenesis'}), ('R_HMR_3907', {'id': 'R_HMR_3907', 'name': '', 'source': 'Human-GEM', 'reactants': ['M_m01796c', 'M_m02554c'], 'products': ['M_m01249c', 'M_m02039c', 'M_m02555c'], 'cpds': ['M_m01796c', 'M_m02554c', 'M_m01249c', 'M_m02039c', 'M_m02555c'], 'EC': 'EC:1.1.1.2', 'pathway': 'Glycolysis / Gluconeogenesis'})]


In [210]:
mcg2rxns = metabolicModels[ 'human_model_mfn' ]['metabolic_rxns']
mcg2rxns[:2]

[{'reactants': ['C02909'],
  'id': 'R06927',
  'source': 'kegg_dre',
  'ecs': ['1.1.1.1'],
  'products': ['C14099'],
  'cpds': ['C02909', 'C14099'],
  'pathway': '1- and 2-Methylnaphthalene degradation'},
 {'reactants': ['C14089'],
  'id': 'R06917',
  'source': 'kegg_dre',
  'ecs': ['1.1.1.1'],
  'products': ['C14090'],
  'cpds': ['C14089', 'C14090'],
  'pathway': '1- and 2-Methylnaphthalene degradation'}]

In [211]:
ATLAS_reactions = list(rxn_dict.values())
ATLAS_reactions[:2]

[{'id': 'R_HMR_3905',
  'name': '',
  'source': 'Human-GEM',
  'reactants': ['M_m01796c', 'M_m02552c'],
  'products': ['M_m01249c', 'M_m02039c', 'M_m02553c'],
  'cpds': ['M_m01796c', 'M_m02552c', 'M_m01249c', 'M_m02039c', 'M_m02553c'],
  'EC': 'EC:1.1.1.1; EC:1.1.1.71',
  'pathway': 'Glycolysis / Gluconeogenesis'},
 {'id': 'R_HMR_3907',
  'name': '',
  'source': 'Human-GEM',
  'reactants': ['M_m01796c', 'M_m02554c'],
  'products': ['M_m01249c', 'M_m02039c', 'M_m02555c'],
  'cpds': ['M_m01796c', 'M_m02554c', 'M_m01249c', 'M_m02039c', 'M_m02555c'],
  'EC': 'EC:1.1.1.2',
  'pathway': 'Glycolysis / Gluconeogenesis'}]

## Save ATLAS_reactions and atlas2020metabolites to Pickle

Note:
* yet to clean up rxns and use IDs without compartments

In [213]:
import pickle

#reactions
with open('ATLAS_reactions.pickle', 'wb') as f:
    pickle.dump(ATLAS_reactions, f, pickle.HIGHEST_PROTOCOL)

In [218]:
#  metabolites
# Better to serialize 
ATLAS_metabolites = []
for C in atlas2020metabolites:
    m = vars(C)
    ATLAS_metabolites.append(m)
    
with open('ATLAS_metabolites_lite.pickle', 'wb') as f:
    pickle.dump(ATLAS_metabolites, f, pickle.HIGHEST_PROTOCOL)

In [219]:
m

{'azimuth_id': 'az_kegg_C17689',
 'name': '',
 'other_ids': {'PubChem': '',
  'KEGG': 'C17689',
  'HMDB': '',
  'ChEBI': '',
  'MetaNetX': '',
  'metabolicATLAS': 'm10042',
  'recon3d': '',
  'charge': -4,
  'formula': 'C45H70N7O19P3S'},
 'formula': 'C45H70N7O19P3S',
 'inchi': '',
 'mw': 1137.3660033233,
 'adducts': {},
 'note': {'atlas2020': ''},
 'charge': -4}

**The pickle data were uploaded to Azimuth via CLI**

In [220]:
!ls


ATLAS_metabolites_lite.pickle	    HMDB-3.5			 __pycache__
ATLAS_metabolites.pickle	    hmdb_metabolites.zip	 pychemy
ATLAS_reactions.pickle		    Human-GEM			 Recon3D
bigg_models_metabolites.txt	    JSON_metabolicModels.py	 refmet.csv
biomodels_from_Xia_site		    mcg1			 refmet.zip
chem_xref.tsv			    microbial_xia		 sample_data
EmoryCIDC_rxns_expt5_C18_neg.txt    old.data_import		 venv
EmoryCIDC_rxns_expt5_HILIC_pos.txt  porting_human_gem2020.ipynb  worm
